In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
dev= pd.read_csv('devices.csv')
tr1= pd.read_csv('transactions_1.csv')
tr2= pd.read_csv('transactions_2.csv')
tr3= pd.read_csv('transactions_3.csv')
users = pd.read_csv('users.csv')
notif = pd.read_csv('notifications.csv')
trans= pd.concat([tr1, tr2, tr3])#all transactions in one dataframe

In [0]:
users['brands']=dev.brand#h sthlh brands apo ta devices is added to users table

In [0]:
users['age']=2018 - users['birth_year']#sthlh age anti gia birth_year

In [0]:
users.plan=users.plan.apply(lambda x: 'PAID' if x!="STANDARD" else x)#h sthlh pairnei pleon standard h paid stoixeia

In [0]:
usd_per_user = pd.pivot_table(data=trans,aggfunc='mean', index='user_id',  values='amount_usd')
users=pd.merge(users , usd_per_user, how='left', left_on='user_id', right_index=True)
###mean amount of usd per user is added to users

In [0]:
number_of_trans=pd.DataFrame(trans.user_id.value_counts())
number_of_trans.rename(columns={'user_id':'amount_of_transactions'},inplace=True)
users=pd.merge(users , number_of_trans, how='left', left_on='user_id', right_index=True)
#amount of transactions per user is added to users table

In [0]:
number_of_notif=pd.DataFrame(notif.user_id.value_counts())
number_of_notif.rename(columns={'user_id':'amount_of_notifications'},inplace=True)
users=pd.merge(users , number_of_notif, how='left', left_on='user_id', right_index=True)

In [186]:
users.head(5)

,user_id,birth_year,country,city,created_date,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,num_referrals,num_successful_referrals,brands,age,amount_usd,amount_of_transactions,amount_of_notifications,BLACK_FRIDAY,BLUE_TUESDAY,ENGAGEMENT_SPLIT_BILL_RESTAURANT,INVEST_IN_GOLD,JOINING_ANNIVERSARY,LOST_CARD_ORDER,MADE_MONEY_REQUEST_NOT_SPLIT_BILL,METAL_RESERVE_PLAN,NO_INITIAL_CARD_ORDER,NO_INITIAL_CARD_USE,ONBOARDING_TIPS_ACTIVATED_USERS,PROMO,PROMO_CARD_ORDER,REENGAGEMENT_ACTIVE_FUNDS,SILVER_ENGAGEMENT_FEES_SAVED,SILVER_ENGAGEMENT_INACTIVE_CARD,WELCOME_BACK,failed_email,failed_push,failed_sms,sent_email,sent_push,sent_sms
0,user_0,1992,ES,Madrid,2018-09-10 18:46:42.843855,0,STANDARD,1.0,1.0,3,0,0,Apple,26,13.068275,545.0,8.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,2.0,0.0
1,user_1,1985,LT,Klaipeda,2018-01-11 18:53:10.531146,1,STANDARD,NaN,NaN,75,0,0,Apple,33,97.000548,146.0,15.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,7.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0,5.0,0.0
2,user_2,1996,IE,Carrickmacross,2018-11-29 17:29:16.879324,0,STANDARD,NaN,NaN,71,0,0,Apple,22,10.579433,282.0,5.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0
3,user_3,1981,IE,Dublin,2018-02-17 11:58:08.319664,0,STANDARD,NaN,NaN,219,0,0,Apple,37,52.345886,158.0,11.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,6.0,0.0
4,user_4,1992,GB,London,2018-01-08 18:40:21.097856,0,STANDARD,NaN,NaN,0,0,0,Apple,26,18.108475,59.0,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,4.0,0.0


In [0]:
notif_reason_name = pd.crosstab(index=notif.user_id, columns=notif.reason)#pinakas me posa notifications ana reason per user
users=pd.merge(users , notif_reason_name, how='left', left_on='user_id', right_index=True)


In [0]:
#amount of  type of notifications, sent/failed per user
notif_channels_channel_sent_failed =pd.crosstab(index=notif.user_id, columns=[notif.status,notif.channel])
notif_channels_channel_sent_failed.columns=['failed_email','failed_push','failed_sms','sent_email', 'sent_push', 'sent_sms']
users=pd.merge(users , notif_channels_channel_sent_failed, how='left', left_on='user_id', right_index=True)


In [0]:
#amount of transactions per type per user
trans_transactions_type = pd.crosstab(trans.user_id, columns=trans.transactions_type)
users=pd.merge(users , trans_transactions_type, how='left', left_on='user_id', right_index=True)

In [189]:
users.shape

(15544, 50)

#tryout to group countries by percentage of paid

In [0]:
countries_anal=pd.crosstab(index=users['country'], columns=users['plan'])


In [0]:
users_countries=users.copy()
countries_anal['countries_paid_percen']=countries_anal['PAID']/(countries_anal['PAID']+countries_anal['STANDARD'])*100

In [0]:
countries_anal['country_per']= countries_anal.countries_paid_percen.apply(lambda x: 0 if x<6  else (1 if x<12  else 2))

In [0]:
users_countries.country.apply(country_to_cat)

In [0]:
def country_to_cat(val):
  if val==countries_anal.index:
       return countries_anal.country_per
x='AU'
country_to_cat(x)

In [206]:
countries_anal.countries_paid_percen

country
AT     5.882353
AU     0.000000
BE     5.309735
BG     2.564103
CH    13.142857
CY     9.740260
CZ     7.665505
DE     8.938547
DK    10.389610
EE     6.666667
ES     5.612245
FI    12.500000
FR     8.903302
GB     7.971303
GG    35.714286
GI    11.428571
GP    22.222222
GR     4.945055
HR    10.204082
HU     6.400000
IE     6.432161
IM    14.285714
IS     0.000000
IT    10.373444
JE    13.333333
LI     0.000000
LT     2.772643
LU    19.047619
LV     9.022556
MQ    33.333333
MT     6.178490
NL     6.060606
NO    17.333333
PL     3.380589
PT     3.490760
RE    28.571429
RO    11.454754
SE    14.423077
SI     5.633803
SK     7.692308
Name: countries_paid_percen, dtype: float64

#Prwth prospa8eia pipeline

##Ta nan mphkan random stis sthles me ta notifications, egine encoding sta ypoloipa

In [0]:
us_1=users.copy()


In [192]:
us_1.isnull().sum()

user_id                                        0
birth_year                                     0
country                                        0
city                                           0
created_date                                   0
user_settings_crypto_unlocked                  0
plan                                           0
attributes_notifications_marketing_push     5260
attributes_notifications_marketing_email    5260
num_contacts                                   0
num_referrals                                  0
num_successful_referrals                       0
brands                                         0
age                                            0
amount_usd                                   532
amount_of_transactions                       532
amount_of_notifications                      388
BLACK_FRIDAY                                 388
BLUE_TUESDAY                                 388
ENGAGEMENT_SPLIT_BILL_RESTAURANT             388
INVEST_IN_GOLD      

In [0]:
us_1=us_1.drop(columns=['num_referrals','num_successful_referrals','birth_year','user_id','city','created_date'])

In [0]:
import random
def change_unknown_device(brand):
  if  brand== 'Unknown':
    return random.sample(['Android', 'Apple'], 1)[0]
  else:
    return brand
    

In [0]:
us_1.brands=us_1.brands.apply(change_unknown_device)

In [125]:
us_1.brands.value_counts()

Android    7818
Apple      7726
Name: brands, dtype: int64

In [0]:
def change_unknown_notif(column_val):
    if np.isnan(column_val) == True: 
        column_val = random.sample([0.0, 1.0], 1)[0]
    else:
         column_val = column_val
    return column_val

In [0]:
us_1.attributes_notifications_marketing_push=us_1.attributes_notifications_marketing_push.apply(change_unknown_notif)
us_1.attributes_notifications_marketing_email=us_1.attributes_notifications_marketing_email.apply(change_unknown_notif)

In [114]:
us_1.attributes_notifications_marketing_email.value_counts()

1.0    11814
0.0     3730
Name: attributes_notifications_marketing_email, dtype: int64

In [115]:
us_1.sample(5)

,country,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,brands,amount_usd,amount_of_transactions,age
7156,IE,0,STANDARD,1.0,1.0,0,Apple,102.623333,12.0,22
3514,PL,0,STANDARD,1.0,1.0,12,Android,1.592500,4.0,28
14579,PT,0,STANDARD,1.0,1.0,33,Apple,22.958733,150.0,50
7803,GB,0,STANDARD,1.0,1.0,3,Android,43.161275,102.0,45
2679,GB,0,STANDARD,1.0,1.0,11,Android,15.812073,357.0,28


In [116]:
us_1.isna().sum()

country                                       0
user_settings_crypto_unlocked                 0
plan                                          0
attributes_notifications_marketing_push       0
attributes_notifications_marketing_email      0
num_contacts                                  0
brands                                        0
amount_usd                                  532
amount_of_transactions                      532
age                                           0
dtype: int64

In [147]:
us_1.fillna(0, inplace=True)
us_1.amount_of_transactions.describe()

count    15544.000000
mean       140.313047
std        238.816473
min          0.000000
25%         14.000000
50%         56.000000
75%        159.000000
max       3381.000000
Name: amount_of_transactions, dtype: float64

In [0]:
us_1.plan

In [128]:
plan_dict = {'STANDARD': 0, 'PAID': 1}
us_1.plan = us_1.plan.map(plan_dict)
us_1.plan

0        0
1        0
2        0
3        0
4        0
        ..
15539    0
15540    0
15541    0
15542    1
15543    0
Name: plan, Length: 15544, dtype: int64

In [148]:
us_1_en=pd.get_dummies(us_1)
us_1_en.sample(2)


,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,age,amount_usd,amount_of_transactions,country_AT,country_AU,country_BE,country_BG,country_CH,country_CY,country_CZ,country_DE,country_DK,country_EE,country_ES,country_FI,country_FR,country_GB,country_GG,country_GI,country_GP,country_GR,country_HR,country_HU,country_IE,country_IM,country_IS,country_IT,country_JE,country_LI,country_LT,country_LU,country_LV,country_MQ,country_MT,country_NL,country_NO,country_PL,country_PT,country_RE,country_RO,country_SE,country_SI,country_SK,brands_Android,brands_Apple
12850,0,0,0.0,1.0,0,26,94.196091,110.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
9654,0,0,1.0,1.0,0,33,4.680000,2.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


##pipeline

In [0]:
X = us_1_en.drop(columns=['plan']).values
X_columns = us_1_en.drop(columns=['plan']).columns
y = us_1_en['plan'].values

In [151]:
us_1_en['plan'].value_counts()

0    14394
1     1150
Name: plan, dtype: int64

In [0]:
from sklearn.model_selection import train_test_split



In [152]:
X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.2, stratify=y)
y_ts.shape

(3109,)

###knn

In [0]:
from imblearn.pipeline import Pipeline 
from sklearn.preprocessing import StandardScaler 
from imblearn.over_sampling import SMOTE 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_recall_fscore_support, classification_report , accuracy_score
from sklearn.neighbors import KNeighborsClassifier 


In [0]:
sampler=SMOTE()
scaler=StandardScaler()
knn=KNeighborsClassifier()

In [154]:
pipe = Pipeline(steps=[('scaler', StandardScaler()),  
                       ('sampler', SMOTE()),
                        
                       ('knn', KNeighborsClassifier())])
param_grid = {'knn__n_neighbors': [3, 5, 7, 9]}
grid = GridSearchCV(pipe, param_grid, cv=2)
grid.fit(X_tr, y_tr)
knn_best_estimator=grid.best_estimator_

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

In [155]:
knn_best_estimator


Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('sampler',
                 SMOTE(k_neighbors=5, kind='deprecated',
                       m_neighbors='deprecated', n_jobs=1,
                       out_step='deprecated', random_state=None, ratio=None,
                       sampling_strategy='auto', svm_estimator='deprecated')),
                ('knn',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,
                                      n_jobs=None, n_neighbors=3, p=2,
                                      weights='uniform'))],
         verbose=False)

In [159]:
preds=grid.predict(X_ts)
print('accuracy: {:.2f}%'.format(accuracy_score(y_ts, preds)*100))
print('\nClassification Report:') 
print(classification_report(y_ts, preds))


accuracy: 81.22%

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.84      0.89      2879
           1       0.19      0.45      0.26       230

    accuracy                           0.81      3109
   macro avg       0.57      0.65      0.58      3109
weighted avg       0.89      0.81      0.85      3109



###DecisionForest


In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import VarianceThreshold

In [0]:
rf = RandomForestClassifier()
var = VarianceThreshold()

In [166]:
pipe2 = Pipeline(steps=[('scaler', StandardScaler()),  
                       ('sampler', SMOTE()),
                        ('selector', var),
                       ('model', rf)])
param_grid2 = {'selector__threshold': [0, 0.2, 0.4],
                'model__n_estimators': [20, 50, 100]}
grid2= GridSearchCV(pipe2, param_grid2, cv=2)
grid2.fit(X_tr, y_tr)
print(grid2.best_params_, '\n')
print(classification_report(y_ts, grid2.predict(X_ts)))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

{'model__n_estimators': 20, 'selector__threshold': 0} 

              precision    recall  f1-score   support

           0       0.95      0.92      0.93      2879
           1       0.26      0.37      0.31       230

    accuracy                           0.88      3109
   macro avg       0.61      0.65      0.62      3109
weighted avg       0.90      0.88      0.89      3109



In [168]:
print('accuracy: {:.2f}%'.format(accuracy_score(y_ts, grid2.predict(X_ts))*100))

accuracy: 87.65%
